In [1]:
import json
from json import JSONDecodeError
import re
import pandas as pd

In [2]:
def json_pd(nome_arquivo):
    """Lê um arquivo JSON e retorna um DataFrame com colunas 'Disease' e 'Symptom'."""
    try:
        with open(nome_arquivo, 'r', encoding='utf-8') as arquivo:
            dados = json.load(arquivo)

        rows = []
        for disease, symptoms in dados.items():  # Iterate over disease-symptoms pairs
            rows.append({"disease": disease, "symptom": symptoms})

        df = pd.DataFrame(rows)
        return df  # Return the DataFrame instead of the original dictionary

    except FileNotFoundError:
        print(f"Erro: Arquivo '{nome_arquivo}' não encontrado.")
        return None
    except json.JSONDecodeError:
        print(f"Erro: Arquivo '{nome_arquivo}' não é um JSON válido.")
        return None

In [3]:
list_files = ['translated_medline.json', 'translated_who.json']

In [4]:
def merge_files(list_files = list_files):
    df1 = json_pd(list_files[0])
    df1['disease'] = df1['disease'].astype(str).str.lower().replace(r'\([^)]*\)', '', regex=True)
    df1['symptom'] = df1['symptom'].astype(str).str.lower().replace(r'\([^)]*\)', '', regex=True)  
    for file in list_files[1:]:  
        df2 = json_pd(file)
        df2['disease'] = df2['disease'].astype(str).str.lower().replace(r'\([^)]*\)', '', regex=True)  
        df2['symptom'] = df2['symptom'].astype(str).str.lower().replace(r'\([^)]*\)', '', regex=True) 
        df1 = pd.merge(df1, df2, on='disease', how='outer')
    return df1

In [5]:
df = merge_files(list_files)


In [6]:
df['symptoms'] = df['symptom_x'].fillna(df['symptom_y'])
df = df.drop(columns=['symptom_x', 'symptom_y'])
df

,disease,symptoms
0,abuso de pessoas idosas,"['lesões físicas', 'mortalidade prematura', 'd..."
1,anemia,"['cansaço tonturas vertigens', 'extremidades f..."
2,artrite reumatoide,"['dor nas articulações', 'rigidez', 'ternura',..."
3,asma,"['tosse persistente, especialmente à noite', '..."
4,borreliose de lyme,['uma erupção vermelha chamada eritema migrató...
...,...,...
95,vírus herpes simplex,"['sem sintomas ou sintomas leves', 'bolhas ou ..."
96,vírus ii t-linfotrópico humano,"['sem sintomas na maioria dos casos', 'nódulos..."
97,vírus nipah,"['febre', 'dores de cabeça', 'dor muscular', '..."
98,zika,"['muitas vezes sem sintomas', 'os sintomas apa..."


In [11]:
# Convert DataFrame to JSON and save to file
df.to_json('doencas.json', orient='records', lines=True)